In [26]:
from agents.rag_agent import chat_agent

In [27]:
# from chains.semantic_search_chunk_chain import chunk_retriever
from chains.semantic_search_chunk_chain import get_chunk_retriever


In [28]:
import pandas as pd
df = pd.read_csv("testset_5files.csv")
question = df['user_input']
ground_truth = df['reference']
retrieval_context = df['reference_contexts']

In [29]:
get_chunk_retriever('openai').invoke("ĐOÀN TNCS HỒ CHÍ MINH là gì?")

[Document(metadata={'link': ['string'], 'page_number': 0, 'filename': ['Quy chế giải thưởng nhà giáo trẻ tiêu biểu - 14.10 conv.pdf']}, page_content='content: BAN CHẤP HÀNH TRUNG ƯƠNG ĐOÀN TNCS HỒ CHÍ MINH\n***\nHà Nội, ngày 18 tháng 10 năm 2019\nQUY CHẾ\nGiải thưởng “Nhà giáo trẻ tiêu biểu” cấp Trung ương\n(Ban hành kèm Quyết định số 1014 -QĐ/TWĐTN-TNT ngày 18 / 10 /2019\ncủa Ban Bí thư Trung ương Đoàn TNCS Hồ Chí Minh)\nCHƯƠNG I\nQUY ĐỊNH CHUNG\nĐiều 1. Tên gọi, số lượng\n1. Tên gọi: Giải thưởng “Nhà giáo trẻ tiêu biểu” do Ban Bí thư Trung ương\nĐoàn Thanh n i ê n C ộ n g s ả n H ồ C h í M i n h t r a o t ặ n g h à n g n ă m v à o d ị p N g à y N h à\ng i á o V i ệ t N a m 2 0 / 1 1 .\n2. Số lượng trao thưởng\nSố lượng cụ thể của từng năm do Hội đồng xét chọn giải thưởng quyết\nđịnh trên cơ sở các ứng cử viên đủ tiêu chuẩn nhưng không quá 100 cá nhân\nđược nhận giải thưởng mỗi năm.\nĐiều 2. Đối tượng xét trao giải thưởng\nGiáo viên, giảng viên có quốc tịch Việt Nam, không quá 35 tuổi

In [30]:
get_chunk_retriever('dvt').invoke("ĐOÀN TNCS HỒ CHÍ MINH là gì?")

No relevant docs were retrieved using the relevance score threshold 0.7


[]

In [31]:
get_chunk_retriever('phobert').invoke("ĐOÀN TNCS HỒ CHÍ MINH là gì?")

[Document(metadata={'link': ['string'], 'page_number': 0, 'filename': ['Quy chế giải thưởng nhà giáo trẻ tiêu biểu - 14.10 conv.pdf']}, page_content='content: BAN CHẤP HÀNH TRUNG ƯƠNG ĐOÀN TNCS HỒ CHÍ MINH\n***\nHà Nội, ngày 18 tháng 10 năm 2019\nQUY CHẾ\nGiải thưởng “Nhà giáo trẻ tiêu biểu” cấp Trung ương\n(Ban hành kèm Quyết định số 1014 -QĐ/TWĐTN-TNT ngày 18 / 10 /2019\ncủa Ban Bí thư Trung ương Đoàn TNCS Hồ Chí Minh)\nCHƯƠNG I\nQUY ĐỊNH CHUNG\nĐiều 1. Tên gọi, số lượng\n1. Tên gọi: Giải thưởng “Nhà giáo trẻ tiêu biểu” do Ban Bí thư Trung ương\nĐoàn Thanh n i ê n C ộ n g s ả n H ồ C h í M i n h t r a o t ặ n g h à n g n ă m v à o d ị p N g à y N h à\ng i á o V i ệ t N a m 2 0 / 1 1 .\n2. Số lượng trao thưởng\nSố lượng cụ thể của từng năm do Hội đồng xét chọn giải thưởng quyết\nđịnh trên cơ sở các ứng cử viên đủ tiêu chuẩn nhưng không quá 100 cá nhân\nđược nhận giải thưởng mỗi năm.\nĐiều 2. Đối tượng xét trao giải thưởng\nGiáo viên, giảng viên có quốc tịch Việt Nam, không quá 35 tuổi

In [32]:
phobert_retriever = get_chunk_retriever(name='dvt')

In [33]:
answers = []
contexts = []
for query in question:
  answers.append(chat_agent.invoke(
    {"input": query},
    {"configurable": {"session_id": 1}}).get("output", "No output text provided."))
  contexts.append([docs.page_content for docs in phobert_retriever.invoke(query)])



> Entering new AgentExecutor chain...
Đoàn TNCS Hồ Chí Minh, hay còn gọi là Đoàn Thanh niên Cộng sản Hồ Chí Minh, là tổ chức chính trị - xã hội của thanh niên Việt Nam, được thành lập vào ngày 26 tháng 3 năm 1931. Tổ chức này có nhiệm vụ tập hợp, đoàn kết và giáo dục thanh niên, đồng thời tham gia vào các hoạt động xã hội, phát triển kinh tế, văn hóa, và bảo vệ Tổ quốc.

Đoàn TNCS Hồ Chí Minh hoạt động dưới sự lãnh đạo của Đảng Cộng sản Việt Nam, với mục tiêu xây dựng thế hệ thanh niên có lý tưởng cách mạng, có trách nhiệm với xã hội và đất nước. Tổ chức này cũng đóng vai trò quan trọng trong việc phát triển phong trào thanh niên, tổ chức các hoạt động tình nguyện, và tham gia vào các chương trình phát triển cộng đồng.

Nếu bạn cần thêm thông tin chi tiết về tổ chức, nhiệm vụ hoặc hoạt động của Đoàn, xin vui lòng cho biết!

> Finished chain.


No relevant docs were retrieved using the relevance score threshold 0.7




> Entering new AgentExecutor chain...

Invoking: `get_chunk_tool` with `{'question': 'Tiêu chuẩn để đạt danh hiệu cấp tỉnh cho học sinh 3 rèn luyện'}`


[Document(metadata={'link': ['string'], 'page_number': 2, 'filename': ['Quy chế giải thưởng nhà giáo trẻ tiêu biểu - 14.10 conv.pdf']}, page_content='content: 1.2. Đối với giáo viên trung học phổ thông\n- Là “Giáo viên dạy giỏi” cấp tỉnh trở lên.\n- Ngoài ra đạt một trong các tiêu chuẩn sau:\n+ Có kết quả nghiên cứu khoa học sư phạm ứng dụng có kèm theo nhận\nxét, đánh giá và xếp loại của nhà trường hoặc của sở giáo dục và đào tạo.\n+ Trực tiếp giảng dạy, bồi dưỡng học sinh tham gia thi học sinh giỏi cấp\ntỉnh, đạt giải từ khuyến khích trở lên.\n1.3. Đối với giáo viên sơ cấp\n- Chủ trì thiết kế và bố trí phòng dạy học chuyên môn hoặc có sáng kiến\nkinh nghiệm về giáo dục nghề nghiệp được nghiệm thu đánh giá đạt yêu cầu.\n- Trực tiếp hướng dẫn học sinh tham gia kỳ thi kỹ năng nghề đạt từ giải ba\ncấp cơ sở (cấp trường) trở lên hoặc th

No relevant docs were retrieved using the relevance score threshold 0.7




> Entering new AgentExecutor chain...

Invoking: `get_chunk_tool` with `{'question': 'Hà Nội có những hoạt động gì cho sinh viên?'}`


[Document(metadata={'link': ['string'], 'page_number': 4, 'filename': ['Quy chế giải thưởng nhà giáo trẻ tiêu biểu - 14.10 conv.pdf']}, page_content='content: - Đạt các huy chương, bằng khen tại các giải thi đấu thể thao, các cuộc thi,\nhội diễn nghệ thuật cấp quốc gia.\n- Nhận được khen thưởng của ngành giáo dục, ngành lao động - thương\nbinh và xã hội từ cấp tỉnh trở lên.\n- Tích cực tham gia các hoạt động hỗ trợ sinh viên (là giáo viên chủ\nnhiệm, cố vấn học tập, mentor hỗ trợ khởi nghiệp, trực tiếp tổ chức tham gia các\nhoạt động tư vấn, hỗ trợ tư vấn định hướng nghề nghiệp, khởi nghiệp,…).\n- Nhà giáo trẻ đang công tác tại vùng sâu, vùng xa, vùng đặc biệt khó\nkhăn, biên giới, hải đảo; đang kiêm nhiệm công tác Đoàn, Hội.\nCHƯƠNG III\nQUY TRÌNH XÉT CHỌN\nĐiều 7. Thẩm quyền đề nghị xét chọn giải thưởng\nBan Thường vụ tỉnh, thành đoàn, đoàn trực thu

...

No relevant docs were retrieved using the relevance score threshold 0.7




> Entering new AgentExecutor chain...

Invoking: `get_chunk_tool` with `{'question': 'HỘI SINH VIÊN VIỆT NAM có những nhiệm vụ trọng tâm nào trong năm học 2024 - 2025 và làm thế nào để tổ chức các hoạt động kỷ niệm 75 năm Ngày truyền thống học sinh, sinh viên?'}`


[Document(metadata={'link': ['string'], 'page_number': 2, 'filename': ['Quy chế giải thưởng nhà giáo trẻ tiêu biểu - 14.10 conv.pdf']}, page_content='content: - Tích cực học tập và làm theo tư tưởng, đạo đức, phong cách Hồ Chí Minh\nthể hiện qua các hành động cụ thể trong giảng dạy, đời sống thường ngày.\n- Không vi phạm Quy định về đạo đức nhà giáo ban hành kèm theo\nQuyết định số 16/2008/QĐ-BGDĐT của Bộ trưởng Bộ Giáo dục và Đào tạo.\n- Nhà giáo trẻ là đoàn viên Đoàn TNCS Hồ Chí Minh, phải được xếp loại\nđoàn viên xuất sắc; nếu là đảng viên, phải là đảng viên hoàn thành tốt nhiệm vụ\ntrở lên; đạt danh hiệu “Lao động tiên tiến” trở lên (tính theo năm liền trước thời\nđiểm xét).\nĐiều 6. Tiêu chuẩn cụ thể\nLưu ý: Tất cả cá

No relevant docs were retrieved using the relevance score threshold 0.7




> Entering new AgentExecutor chain...

Invoking: `get_chunk_tool` with `{'question': 'HỘI SINH VIÊN VIỆT NAM có những quy định gì về tiêu chuẩn xét chọn danh hiệu “Sinh viên 5 tốt” và làm thế nào để các Hội Sinh viên cấp tỉnh, cấp trường có thể tham gia vào quá trình này?'}`


[Document(metadata={'link': ['string'], 'page_number': 2, 'filename': ['Quy chế giải thưởng nhà giáo trẻ tiêu biểu - 14.10 conv.pdf']}, page_content='content: - Tích cực học tập và làm theo tư tưởng, đạo đức, phong cách Hồ Chí Minh\nthể hiện qua các hành động cụ thể trong giảng dạy, đời sống thường ngày.\n- Không vi phạm Quy định về đạo đức nhà giáo ban hành kèm theo\nQuyết định số 16/2008/QĐ-BGDĐT của Bộ trưởng Bộ Giáo dục và Đào tạo.\n- Nhà giáo trẻ là đoàn viên Đoàn TNCS Hồ Chí Minh, phải được xếp loại\nđoàn viên xuất sắc; nếu là đảng viên, phải là đảng viên hoàn thành tốt nhiệm vụ\ntrở lên; đạt danh hiệu “Lao động tiên tiến” trở lên (tính theo năm liền trước thời\nđiểm xét).\nĐiều 6. Tiêu chuẩn cụ thể\nLưu 

No relevant docs were retrieved using the relevance score threshold 0.7


In [34]:
df["response"] = answers
df["retrieved_contexts"] = contexts
df.to_csv("test_set_rag_dvt_13-12.csv")

In [35]:
import ast

data = {
    "user_input": question,
    "answer": answers,
    "contexts": contexts,
    "reference": df['reference'],
    "reference_contexts": df["reference_contexts"].apply(ast.literal_eval)}

In [37]:
from ragas.dataset_schema import EvaluationDataset
# from ragas import 
from datasets import Dataset
import pandas as pd
import ast
df = pd.read_csv('test_set_rag_dvt_13-12.csv')
df['retrieved_contexts'] = df['retrieved_contexts'].apply(ast.literal_eval)
df['reference_contexts'] = df['reference_contexts'].apply(ast.literal_eval)

dataset = Dataset.from_pandas(df)
# dataset= Dataset.from_dict(**data)

In [38]:
from ragas import evaluate
from ragas.cost import get_token_usage_for_openai

from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset = dataset, 
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
    token_usage_parser=get_token_usage_for_openai,
)

df = result.to_pandas()

Evaluating: 100%|██████████| 60/60 [01:54<00:00,  1.92s/it]


In [39]:
print(result.total_tokens())


input_tokens=174293 output_tokens=28094 model=''


In [40]:
df.to_csv("final_result_dvt.csv")